In [1]:
%reset -f

### (A) Create all digit sequences with a specific numeric distance
Here, all possible <b>unique</b> combinations of digits are created. All adjacent digits in ordered sequence (either increasing or decreasing) are separated by numeric disctance specified in `numdistances` (here 2 or 3). Digits are chosen from list `digits` (here all digits are considered, with zero as exception). Finally seqences are stored in:
- `seqIn` list of increasing (ordered) sequences (<b>12 total</b>)
- `seqDe` list of decreasing (ordered) sequences (<b>12 total</b>)
- `seqNo` list of non-ordered sequences (<b>48 total</b>)

Non-ordered sequences are constructed from increasing and decreasing sequences by permuting first two items or last two items. This procedure yields four non-ordered sequences for one pair of increasing/decreasing sequence.

In [2]:
from itertools import product

#--- set available digits
digits = [1, 2, 3, 4, 5, 6, 7, 8, 9]
#--- set available numeric distances
numdistances = [2, 3]

#--- create all increasing sequences 
seqIn = []
for ndpair in product(numdistances, repeat=2):
    seqIn.extend([[dig, dig+ndpair[0], dig+ndpair[0]+ndpair[1]] for dig in digits if dig+ndpair[0]+ndpair[1] in digits])
#--- create all decreasing sequences
seqDe = [seq[::-1] for seq in seqIn]
#--- create all non-ordered sequences
seqNo = []
for seqpair in zip(seqIn, seqDe):
    seqNo.extend([[seqpair[0][0], seqpair[0][2], seqpair[0][1]],
                  [seqpair[0][1], seqpair[0][0], seqpair[0][2]],
                  [seqpair[1][0], seqpair[1][2], seqpair[1][1]],
                  [seqpair[1][1], seqpair[1][0], seqpair[1][2]]])

### (B) Divide stimuli list with respect to target inclusion
This part of script specify target digit in control condition (where subject is asked to determine if target digit is present within a sequence or not). Target digit is specified in `target` variable. Here digit $4$ was chosen, due to its high occurrence frequency among all sequences (<b>7/16</b> in ordered sequences and <b>28/64</b> in non-ordered sequences). With digit $4$ it is possible to counterbalance presented sequences with respect to target occurrence.

Three sequence groups are further divided into six groups – three containing target digit and three without target digit in a sequence. Groups are named:
- `seqInTp` list of increasing (ordered) sequence containing target (<b>7 total</b>)
- `seqInTn` list of increasing (ordered) sequence not containing target (<b>9 total</b>)
- `seqDeTp` list of decreasing (ordered) sequence containing target (<b>7 total</b>)
- `seqDeTn` list of decreasing (ordered) sequence not containing target (<b>9 total</b>)
- `seqNoTp` list of non-ordered sequence containing target (<b>28 total</b>)
- `seqNoTn` list of non-ordered (ordered) sequence not containing target (<b>36 total</b>)

In [3]:
#--- set target number
target = 4

#--- divide sequences
seqInTp = [seq for seq in seqIn if target in seq]
seqInTn = [seq for seq in seqIn if target not in seq]
seqDeTp = [seq for seq in seqDe if target in seq]
seqDeTn = [seq for seq in seqDe if target not in seq]
seqNoTp = [seq for seq in seqNo if target in seq]
seqNoTn = [seq for seq in seqNo if target not in seq]    

### (C) Create final stimuli lists
This part of script draw sequences at random from appropriate lists. `N_block` sets of sequences (here 4) are created (one set per experimental block) with `N_seq` trials/block (12 trials/block). Types of sequences are counterbalanced so that <i>each block contains</i>:
- 6 non-ordered sequences (3 with target and 3 w/o target)
- 6 ordered sequences including:
    - 3 increasing sequences (2 (or 1) with target and 1 (or 2) w/o target)
    - 3 decreasing sequences (1 (or 2) with target and 2 (or 1) w/o target)

Sequences are randomly shuffled before they are used for final sets. All sequences are unique, meaning that through entire experiment each sequence is always displayed twice (once for each task condition). Additional data is stored for further use in PsychoPy scripts and output logs:
- `stimOrdr` contains information about type of sequence (increasing=1, decreasing=-1, non-ordered=0)
- `stimTarg` contains information about target within a sequence (present=1, absent=0)
- `stimNumd` contains information about total numeric distance for a given sequence (4, 5 or 6)

In [4]:
from random import shuffle

#--- number of blocks & sequences per block
N_block = 4
N_seq = 12

def addtostim(seqList, block, num_stim, isOrder, isTarget):
    for i in range(num_stim):
        seq = seqList.pop()
        stim[block].append(seq)
        stimOrdr[block].append(isOrder)
        stimTarg[block].append(isTarget)
        stimNumd[block].append(numdist(seq))
def numdist(seq):
    return sorted(seq)[2]-sorted(seq)[0]

#--- shuffle lists
shuffle(seqInTp)
shuffle(seqInTn)
shuffle(seqDeTp)
shuffle(seqDeTn)
shuffle(seqNoTp)
shuffle(seqNoTn)

#--- draw stimuli
stim = [[],[],[],[]]
stimOrdr = [[],[],[],[]] # is sequence ordered? (1=increasing, -1=decreasing, 0=non-ordered)
stimTarg = [[],[],[],[]] # does sequence contain target?
stimNumd = [[],[],[],[]] # total numeric distance
for block in range(N_block):
    #--- Non-ordered
    addtostim(seqNoTp, block, num_stim=3, isOrder=0, isTarget=1) # with target
    addtostim(seqNoTn, block, num_stim=3, isOrder=0, isTarget=0) # with no target
    #--- Ordered
    addtostim(seqInTp, block, num_stim=(1 + (block+1) % 2), isOrder=1,  isTarget=1) # with target & increasing
    addtostim(seqDeTp, block, num_stim=(1 + block % 2),     isOrder=-1, isTarget=1) # with target & decreasing
    addtostim(seqInTn, block, num_stim=(1 + block % 2),     isOrder=1,  isTarget=0) # with no target & increasing
    addtostim(seqDeTn, block, num_stim=(1 + (block+1) % 2), isOrder=-1, isTarget=0) # with no target & decreasing

### (D) Concatenate blocks and save to one xlsx file
Finally, sequences are concatenated into one table (Pandas dataframe) and saved as .xlsx spreadsheet under name specified in `stim_filename`. Additionaly, table is displayed within Notebook (see below).

In [5]:
import pandas as pd

#--- set filename
stim_filename = 'stimuli.xlsx'

#--- concatenate variables for all blocks
blockn = [n for n in range (1,N_block+1) for rep in range(N_seq)]   # set (block) number
digitL = [seq[0] for stimblock in stim for seq in stimblock]        # left digit
digitC = [seq[1] for stimblock in stim for seq in stimblock]        # central digit
digitR = [seq[2] for stimblock in stim for seq in stimblock]        # right digit
isOrder = [value for stimblock in stimOrdr for value in stimblock]  # is sequence ordered?
isTarget = [value for stimblock in stimTarg for value in stimblock] # does it contain target?
numDist = [value for stimblock in stimNumd for value in stimblock]  # total numeric distance

#--- create dictionary
d = {'block': blockn, 'digitL': digitL, 'digitC': digitC, 'digitR': digitR,
     'isOrder': isOrder, 'isTarget': isTarget, 'numDist': numDist}
#--- convert to dataframe
df = pd.DataFrame(data=d)

#--- save stimulis
writer = pd.ExcelWriter(stim_filename, engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1', index=False)
writer.save()

#--- show df
digitLCR = [seq for stimblock in stim for seq in stimblock]
dshow = {'block': blockn, 'sequence': digitLCR,
         'isOrder': isOrder, 'isTarget': isTarget, 'numDist': numDist}
df2 = pd.DataFrame(data=dshow)
print(df2)

/home/connectomics/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/connectomics/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


    block   sequence  isOrder  isTarget  numDist
0       1  [7, 4, 9]        0         1        5
1       1  [4, 2, 7]        0         1        5
2       1  [4, 6, 1]        0         1        5
3       1  [6, 3, 9]        0         0        6
4       1  [5, 2, 7]        0         0        5
5       1  [3, 8, 5]        0         0        5
6       1  [2, 4, 6]        1         1        4
7       1  [4, 6, 8]        1         1        4
8       1  [9, 7, 4]       -1         1        5
9       1  [5, 7, 9]        1         0        4
10      1  [6, 3, 1]       -1         0        5
11      1  [9, 6, 3]       -1         0        6
12      2  [7, 2, 4]        0         1        5
13      2  [4, 7, 1]        0         1        6
14      2  [7, 9, 4]        0         1        5
15      2  [8, 2, 5]        0         0        6
16      2  [5, 7, 2]        0         0        5
17      2  [3, 5, 1]        0         0        4
18      2  [2, 4, 7]        1         1        5
19      2  [8, 6, 4]